In [ ]:
workdir_ = '/glade/work/juliob/'
import sys
sys.path.append(workdir_ + 'myPythonTools/GenlTools/')
sys.path.append(workdir_ + 'myPythonTools/Utils/')
sys.path.append(workdir_ + 'PyRegridding/Regridder/')

# Own local packages
import AveragingUtils as Av
import VertRegridFlexLL as Vrg
import PlotUtil as Pu

# The usual
from datetime import date
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.tri as tri

# Cartopy for pretty maps
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Some other useful packages 
import importlib
import copy
import time
import cftime

importlib.reload( Pu )
importlib.reload(Av)

In [ ]:
#####

path_C ='/glade/derecho/scratch/juliob/archive/FMTHIST_01/atm/hist/FMTHIST_01.cam.h1.*.nc'
Xc = xr.open_mfdataset( path_C ,data_vars='different', coords='different' )
path_X ='/glade/derecho/scratch/juliob/archive/nubr_FMTHIST_01/atm/hist/nubr_FMTHIST_01.cam.h1.*.nc'
Xx = xr.open_mfdataset( path_X ,data_vars='different', coords='different' )

#D2b = xr.open_mfdataset( '/glade/derecho/scratch/juliob/archive/test02/atm/hist/test02.cam.h0.*.nc' , combine='by_coords', data_vars='minimal', coords='minimal')
#D2c = xr.open_mfdataset( '/glade/derecho/scratch/juliob/archive/test02/atm/hist/test02.cam.h0.*.nc' , combine='by_coords', join='exact' )
#D2c = xr.open_mfdataset( '/glade/derecho/scratch/juliob/archive/test02/atm/hist/test02.cam.h0.*.nc' , combine='by_coords', coords='minimal')

#concat_dim=None )


In [ ]:
Xx.UBM_MOVMTN

In [ ]:
##
#ubm_m = 86400. * Xx.UBM_MOVMTN[20,:,:].values

ubm_m_c = Xc.UBM_MOVMTN[240,:,:].values
ubm_m_x = Xx.UBM_MOVMTN[240,:,:].values
lon=Xx.lon.values
lat=Xx.lat.values
lev=Xx.lev.values



In [ ]:
plt.plot( Xx.UBM_MOVMTN[:,4,:].values.flatten())

In [ ]:
#plt.plot( ubm_m_c.flatten())
plt.plot( ubm_m_x.flatten() - ubm_m_c.flatten() )


In [ ]:
print(lev[93-45])
zlev=-7.0*np.log( lev/1_000. )
print( zlev[93-45] )

In [ ]:
Dx['PRECT'] = Dx['PRECC']+Dx['PRECL']
new_attributes = {
    'units': 'ms-1',
    'description': 'sum of PRECC and PRECL',
}
Dx['PRECT'].attrs.update(new_attributes)

season='son'
print( 'PRECT' not in Dx )

UUc_s = Av.SeasonalZonal( ds=Dc, season=season , fld='U')
UUx_s = Av.SeasonalZonal( ds=Dx, season=season , fld='U')


In [ ]:
Dx['PRECL']

In [ ]:
fig = plt.figure(figsize=(20, 5))
clev=np.linspace( -60,140,num=21) 
dlev=np.linspace( -20,20,num=21) 
cmap='gist_ncar'

Axes1 = Pu.axes_def(n=1,nxplo=3,nyplo=1 ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat,zlev,UUc_s ,levels=clev, cmap=cmap )
co2=ax1.contour(lat,zlev,UUc_s ,levels=clev, colors='black')
ax1.set_title(f"Control <{exp_C}> {season.upper()}" )
cb=plt.colorbar(co1)

Axes1 = Pu.axes_def(n=2,nxplo=3,nyplo=1 ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat,zlev,UUx_s ,levels=clev, cmap=cmap )
co2=ax1.contour(lat,zlev,UUx_s ,levels=clev, colors='black')
ax1.set_title(f"Test <{exp_X}> {season.upper()}" )
cb=plt.colorbar(co1)

Axes1 = Pu.axes_def(n=3,nxplo=3,nyplo=1 ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat,zlev,UUx_s-UUc_s ,levels=dlev, cmap='bwr' )
co2=ax1.contour(lat,zlev,UUx_s-UUc_s ,levels=dlev, colors='black')
ax1.set_title('Diff '+season.upper()  )
cb=plt.colorbar(co1)



In [ ]:
clev=np.linspace( -60,140,num=21) 
plt.contourf(lat,zlev,UUx_s ,levels=clev )
plt.colorbar()

In [ ]:
plt.contourf(lat,zlev,UUc_s ,levels=clev )
plt.colorbar()

In [ ]:
print(np.shape(u10_c))
print(np.shape(u10_x))
print(np.shape(lon))
print(np.shape(lat))

print(Dc.lev.values[20])

In [ ]:
fig = plt.figure(figsize=(15, 5))
clev=np.linspace(0,10,num=16)
dlev=np.linspace(-1,1,num=11) * 10

clev=np.linspace(-20,50,num=16)


Axes1 = Pu.axes_def(n=1,nxplo=3,nyplo=1 ) 
ax1 = fig.add_axes( Axes1 )
co1 = ax1.tricontourf( lon,lat,u10_c.mean(axis=0)  ,levels=clev , cmap='gist_ncar' ) 
pr1 = ax1.tricontour( lon,lat,ps[0,:],levels=[10_000,100_100],colors='black')
ax1.set_title('Control')
cb=plt.colorbar(co1)

Axes1 = Pu.axes_def(n=2,nxplo=3,nyplo=1 ) 
ax1 = fig.add_axes( Axes1 )
co1 = ax1.tricontourf( lon,lat,u10_x.mean(axis=0)  ,levels=clev, cmap='gist_ncar' ) 
pr1 = ax1.tricontour( lon,lat,ps[0,:],levels=[10_000,100_100],colors='black')
ax1.set_title('Test')
cb=plt.colorbar(co1)

Axes1 = Pu.axes_def(n=3,nxplo=3,nyplo=1 ) 
ax1 = fig.add_axes( Axes1 )
co1 = ax1.tricontourf( lon,lat,u10_x.mean(axis=0)-u10_c.mean(axis=0)  ,levels=dlev, cmap='bwr' ) 
pr1 = ax1.tricontour( lon,lat,ps[0,:],levels=[10_000,100_100],colors='black')
ax1.set_title('Test - Control')
cb=plt.colorbar(co1)
